# Modelling CIFAR10 Dataset Using Resnet18

### The requirements in detail:
  
1. Go through this repository: https://github.com/kuangliu pytorch-cifar
2. Extract the ResNet18 model from this repository and add it to your API/repo. 
3. Use your data loader, model loading, train, and test code to train ResNet18 on Cifar10
4. Your Target is 85% accuracy. No limit on the number of epochs. 5. Use default ResNet18 code (so params are fixed). 
6. Once done finish S8-Assignment-Solution. 



### The solution:
- Using resnet18, the model could achieve 82% easily within 15 epochs
- The model started struggling after that with drop in accuracy and hovering mostly about 83% 
- After 23rd epoch, learning rate was reduced and weight decay of 1e-5 was added. 
- The model quickly jumped to 85.5% with this change in the next epoch itself, but then stayed there for the next 18 epochs

### Issues:
- The model started overfitting from 6th epochs onwards and by the 42nd, it was heavily overfitting with training accuracy reaching >98%
- The model was trained totally for 42 epochs, but most of the epochs didnt help improve the accuracy that much
- We didnt use learning rates optimally to make the training mroe efficient


## Setup google drive for storage & retrieval

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

module_folder = 'drive/My Drive/eva_stored_from_colab/eva5/cifar10_resnet/'
github_raw_models_path = "tsai_projects/assignment8_cifar10_resnet/raw_models"
github_custom_utils_path = "tsai_projects/assignment8_cifar10_resnet/custom_utils"


base_folder = module_folder+'trained_weights/'
anlt_folder = module_folder+'analysis_materials/'
acc_recorder_files = ["maxAcc_cifar10_resnet"]
model_file_names = ["mdl_cifar10_resnet"]

Mounted at /content/drive


## Get module from github & set it up for use

In [ ]:
# !git clone  https://github.com/sairamsubramaniam/tsai_projects.git

# import os
# from pathlib import Path
# import shutil
# import time

# raw_models_dir = Path(module_folder + "raw_models/")
# custom_utils_dir = Path(module_folder + "custom_utils/")

# if raw_models_dir.exists():
#     shutil.rmtree(raw_models_dir)

# if custom_utils_dir.exists():
#     shutil.rmtree(custom_utils_dir)

# time.sleep(60)

# shutil.move(github_raw_models_path, module_folder)
# shutil.move(github_custom_utils_path, module_folder)

# shutil.rmtree('tsai_projects/')

# try:
#     os.mkdir(base_folder)
# except:
#     pass

# try:
#     os.mkdir(anlt_folder)
# except:
#     pass

# time.sleep(60)

Cloning into 'tsai_projects'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 121 (delta 66), reused 95 (delta 43), pack-reused 0
Receiving objects: 100% (121/121), 608.88 KiB | 7.00 MiB/s, done.
Resolving deltas: 100% (66/66), done.


## Import all the required modules including the custom ones

In [2]:
from __future__ import print_function
from operator import itemgetter
import time
import sys
sys.path.append(module_folder)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
%matplotlib inline

from raw_models import resnets
import custom_utils
from custom_utils import helpers
from custom_utils import analytics

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## Download Data

In [3]:
torch.manual_seed(1)
batch_size = 128

transform = transforms.Compose(
                          [transforms.RandomRotation(10) ,
                           transforms.RandomHorizontalFlip(0.25) ,
                           transforms.ToTensor(),
                          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Check the number of parameters

In [ ]:
!pip install torchsummary
from torchsummary import summary
summary(resnets.ResNet18().to(device), input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

## Train - Round 1 (4 epochs until Power outage)

In [ ]:
fn_idx = 0
epochs = 26

acc_recorder_file = acc_recorder_files[fn_idx]
model_file_name = model_file_names[fn_idx]

accuracy_store_path = base_folder+acc_recorder_file
model_sd_save_path = base_folder+model_file_name+"_sd.pth"

model = resnets.ResNet18().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=0, momentum=0.9)
loss_func = F.cross_entropy


trial1 = helpers.train_epochs(
                    model=model, 
                    device=device, 
                    train_loader=train_loader,
                    test_loader=test_loader, 
                    optimizer=optimizer, 
                    loss_func=loss_func, 
                    epochs=epochs,
                    accuracy_store_path=accuracy_store_path, 
                    model_sd_save_path=model_sd_save_path,
                    save_if_better_acc=True)

  0%|          | 0/391 [00:00<?, ?it/s]


LAST RECORDED MAX ACCURACY:  0.0

EPOCH NUM 1


loss=1.9674272537231445 batch_id=390: 100%|██████████| 391/391 [03:08<00:00,  2.08it/s]


Train Data: Average loss: 0.0185, Accuracy: 10002/50000 (20.0040%)




Test set: Average loss: 1.9513, Accuracy: 2572/10000 (25.7200%)

-----------------------------------------------

EPOCH NUM 2


...


Train Data: Average loss: 0.0111, Accuracy: 23959/50000 (47.9180%)




Test set: Average loss: 1.3673, Accuracy: 4936/10000 (49.3600%)



  0%|          | 0/391 [00:00<?, ?it/s]

-----------------------------------------------

EPOCH NUM 5


loss=1.380722165107727 batch_id=215:  55%|█████▌    | 216/391 [01:44<01:27,  2.01it/s]

## Train - Round 2 (19 epochs more)

In [ ]:
fn_idx = 0
epochs = 20

acc_recorder_file = acc_recorder_files[fn_idx]
model_file_name = model_file_names[fn_idx]

accuracy_store_path = base_folder+acc_recorder_file
model_sd_save_path = base_folder+model_file_name+"_sd.pth"

model = resnets.ResNet18().to(device)
model.load_state_dict(torch.load(model_sd_save_path))

optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=0, momentum=0.9)
loss_func = F.cross_entropy


trial1 = helpers.train_epochs(
                    model=model, 
                    device=device, 
                    train_loader=train_loader,
                    test_loader=test_loader, 
                    optimizer=optimizer, 
                    loss_func=loss_func, 
                    epochs=epochs,
                    accuracy_store_path=accuracy_store_path, 
                    model_sd_save_path=model_sd_save_path,
                    save_if_better_acc=True)

  0%|          | 0/391 [00:00<?, ?it/s]


LAST RECORDED MAX ACCURACY:  0.6224

EPOCH NUM 1


loss=0.7898316383361816 batch_id=390: 100%|██████████| 391/391 [03:13<00:00,  2.02it/s]


Train Data: Average loss: 0.0072, Accuracy: 33692/50000 (67.3840%)




Test set: Average loss: 0.8929, Accuracy: 6761/10000 (67.6100%)



...


  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 0.5907, Accuracy: 8288/10000 (82.8800%)

-----------------------------------------------

EPOCH NUM 19


loss=0.16025729477405548 batch_id=390: 100%|██████████| 391/391 [03:13<00:00,  2.02it/s]


Train Data: Average loss: 0.0013, Accuracy: 47102/50000 (94.2040%)




Test set: Average loss: 0.6088, Accuracy: 8299/10000 (82.9900%)

-----------------------------------------------
TOTAL TRAINING TIME:  3804.6582436561584
LAST 10 EPOCH AVG ACC:  0.82208
LAST 5 EPOCH AVG ACC:  0.8290200000000001
MAX ACCURACY:  0.8304


## Train - Round 3 (adding regularizers and reducing learning rate)

In [4]:
fn_idx = 0
epochs = 20

acc_recorder_file = acc_recorder_files[fn_idx]
model_file_name = model_file_names[fn_idx]

accuracy_store_path = base_folder+acc_recorder_file
model_sd_save_path = base_folder+model_file_name+"_sd.pth"

model = resnets.ResNet18().to(device)
model.load_state_dict(torch.load(model_sd_save_path))

optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-5, momentum=0.9)
loss_func = F.cross_entropy


trial1 = helpers.train_epochs(
                    model=model, 
                    device=device, 
                    train_loader=train_loader,
                    test_loader=test_loader, 
                    optimizer=optimizer, 
                    loss_func=loss_func, 
                    epochs=epochs,
                    accuracy_store_path=accuracy_store_path, 
                    model_sd_save_path=model_sd_save_path,
                    save_if_better_acc=True)

  0%|          | 0/391 [00:00<?, ?it/s]


LAST RECORDED MAX ACCURACY:  0.8304

EPOCH NUM 1


loss=0.0975339338183403 batch_id=390: 100%|██████████| 391/391 [00:30<00:00, 13.02it/s]


Train Data: Average loss: 0.0010, Accuracy: 47871/50000 (95.7420%)




  0%|          | 0/391 [00:00<?, ?it/s]

...


  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 0.6358, Accuracy: 8547/10000 (85.4700%)

-----------------------------------------------

EPOCH NUM 19


loss=0.030765632167458534 batch_id=390: 100%|██████████| 391/391 [00:29<00:00, 13.04it/s]


Train Data: Average loss: 0.0003, Accuracy: 49409/50000 (98.8180%)




Test set: Average loss: 0.6268, Accuracy: 8559/10000 (85.5900%)

-----------------------------------------------
TOTAL TRAINING TIME:  627.9362323284149
LAST 10 EPOCH AVG ACC:  0.8561400000000001
LAST 5 EPOCH AVG ACC:  0.8552799999999999
MAX ACCURACY:  0.8593
